## 查询参数校验

In [1]:
import sys
print(sys.version)

3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


Python版本是3.10.5。让我们以下面的应用程序为例：

In [4]:
import uvicorn
from fastapi import FastAPI
app = FastAPI()
@app.get("/items/")
async def read_items(q: str | None = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [8684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:62909 - "GET /items/?q=test HTTP/1.1" 200 OK
INFO:     127.0.0.1:62946 - "GET /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8684]


如果在浏览器中输入 http://127.0.0.1:8009/items/?q=test

会看到 {"items":[{"item_id":"Foo"},{"item_id":"Bar"}],"q":"test"}

查询参数 q 的类型为 str，默认值为 None，因此它是可选的。

如果在浏览器中输入 http://127.0.0.1:8009/items/

会看到 {"items":[{"item_id":"Foo"},{"item_id":"Bar"}]}

## 额外的校验

我们打算添加约束条件：即使 q 是可选的，但只要提供了该参数，则该参数值不能超过5个字符的长度。

为此，首先从 fastapi 导入 Query：

现在，将 Query 用作查询参数的默认值，并将它的 max_length 参数设置为 5：

In [6]:
import uvicorn
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/items/")
async def read_items(q: str | None = Query(default=None, max_length=5)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [8684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:63000 - "GET /items/?q=testdata HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8684]


如果在浏览器中输入 http://127.0.0.1:8009/items/?q=testdata

会看到 {"detail":[{"type":"string_too_long","loc":["query","q"],"msg":"String should have at most 5 characters","input":"testdata","ctx":{"max_length":5},"url":"https://errors.pydantic.dev/2.5/v/string_too_long"}]}

由于我们必须用 Query(default=None) 替换默认值 None，Query 的第一个参数同样也是用于定义默认值。

然后，我们可以将更多的参数传递给 Query。

## 添加更多校验

你还可以添加 min_length 参数：

In [ ]:
import uvicorn
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/items/")
async def read_items(q: str | None = Query(default=None, min_length=3, max_length=5)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

## 添加正则表达式

你可以定义一个参数值必须匹配的正则表达式：

In [8]:
import uvicorn
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/items/")
async def read_items(q: str | None = Query(default=None, min_length=3, max_length=5, pattern="^test.?$")):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [8684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:63087 - "GET /items/?q=tests HTTP/1.1" 200 OK
INFO:     127.0.0.1:63088 - "GET /items/?q=testa HTTP/1.1" 200 OK
INFO:     127.0.0.1:63091 - "GET /items/?q=testb HTTP/1.1" 200 OK
INFO:     127.0.0.1:63093 - "GET /items/?q=text HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:63094 - "GET /items/?q=testab HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8684]


这个指定的正则表达式通过以下规则检查接收到的参数值：

^：以该符号之后的字符开头，符号之前没有字符testery: 值精确地teste
.?：零个或一个任意字符ry。
$: 到ery 之后没有更

根据上述规则，test、tests、testa、testb、testc、testd都是允许的字符。
testab、testabc、text则不是被允许的。多字符。

## 声明为必需参数

当我们不需要声明额外的校验或元数据时，只需不声明默认值就可以使 q 参数成为必需参数，例如：

q: str

因此，当你在使用 Query 且需要声明一个值是必需的时，只需不声明默认参数：

In [9]:
import uvicorn
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/items/")
async def read_items(q: str = Query(min_length=3, max_length=5, pattern="^test.?$")):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [8684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:63271 - "GET /items/ HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8684]


如果在浏览器中输入 http://127.0.0.1:8009/items/

会看到 {"detail":[{"type":"missing","loc":["query","q"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.5/v/missing"}]}

## 使用省略号(...)声明必需参数

有另一种方法可以显式的声明一个值是必需的，即将默认参数的默认值设为 ... ：

这将使 FastAPI 知道此查询参数是必需的。

请记住，在大多数情况下，当你需要某些东西时，可以简单地省略 default 参数，因此你通常不必使用 ... 

In [ ]:
import uvicorn
from fastapi import FastAPI, Query
app = FastAPI()
@app.get("/items/")
async def read_items(q: str = Query(default=..., min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

## 声明更多元数据

你可以添加更多有关该参数的信息。
你可以添加 title：

Query(default=None, title="Query string", min_length=3)

以及 description：

Query(default=None, title="Query string", description="Query string for the items to search in the database", min_length=3)